In [1]:
import tensorflow as tf
import keras
from keras import layers

2023-08-21 16:36:39.298828: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-21 16:36:39.299921: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 16:36:39.324737: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-21 16:36:39.325333: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 16:36:39.931665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [47]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"), # After this step the matrix is 3x2
        layers.Dense(3, activation="relu", name="layer2"), # After this step the matrix is 3x3 (3x2 * 2x3)
        layers.Dense(4, name="layer3"), # After this step the matrix is 3x4 (3x3 * 3x4)
    ]
)

x = tf.ones((3, 3))
y = model(x)
#print(y.numpy())
for layer in model.layers:
    print(layer.name)
    weights, biases = layer.get_weights()
    print("Weights:", weights)
    print("Biases:", biases)
    print("------------")


layer1
Weights: [[-1.0493544   0.7363399 ]
 [-0.19919515  0.207973  ]
 [-0.18664336 -0.84004706]]
Biases: [0. 0.]
------------
layer2
Weights: [[-0.21164376 -0.7095196  -0.45058084]
 [-0.7338321   0.1422962  -0.51181006]]
Biases: [0. 0. 0.]
------------
layer3
Weights: [[-0.369933   -0.89441603  0.08582222 -0.8300144 ]
 [ 0.9001694   0.08848596  0.32949078  0.28434944]
 [ 0.23469234  0.629735    0.3881148   0.35783112]]
Biases: [0. 0. 0. 0.]
------------


In [48]:
# List all the layers in the model.
model.layers

In [52]:
# Generally, all layers in Keras need to know the shape of their inputs 
# in order to be able to create their weights. So when you create a layer like this, 
# initially, it has no weights:
layer = layers.Dense(2)
print(layer.weights)  # Empty


[]


In [53]:
x = tf.ones((3, 3))
y = layer(x)
layer.weights

[<tf.Variable 'dense_1/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.76193297, -0.9696126 ],
        [-0.08693612,  0.9868624 ],
        [-0.2670561 , -0.71430725]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [56]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()
    

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Number of weights after calling the model: 6


In [57]:
model.summary()

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (1, 2)                    10        
                                                                 
 dense_9 (Dense)             (1, 3)                    9         
                                                                 
 dense_10 (Dense)            (1, 4)                    16        
                                                                 
Total params: 35 (140.00 Byte)
Trainable params: 35 (140.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
# More complex model
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 40, 40, 32)        0         
 D)                                                              
                                                                 
Total params: 11680 (45.62 KB)
Trainable params: 11680 (45.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432 